In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# Step 2: Create Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
data_dir = '/content/drive/MyDrive/Colab Notebooks/Railway-Track-Surface-Faults-Dataset'

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 5153 images belonging to 7 classes.


In [5]:

# Step 3: Load Pre-trained VGG16 Model
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

vgg_base.trainable=False

58889256/58889256 [==============================] - 0s 0us/step


In [6]:
# Step 4: Add Custom Classifier
x = Flatten()(vgg_base.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)

model = Model(inputs=vgg_base.input, outputs=predictions)

In [7]:
# Step 5: Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10
)

Epoch 1/10
161/161 [==============================] - 1746s 11s/step - loss: 1.5897 - accuracy: 0.6038
Epoch 2/10
161/161 [==============================] - 99s 616ms/step - loss: 0.9046 - accuracy: 0.6413
Epoch 3/10
161/161 [==============================] - 101s 629ms/step - loss: 0.8144 - accuracy: 0.6922
Epoch 4/10
161/161 [==============================] - 99s 612ms/step - loss: 0.7835 - accuracy: 0.7032
Epoch 5/10
161/161 [==============================] - 99s 612ms/step - loss: 0.7631 - accuracy: 0.7112
Epoch 6/10
161/161 [==============================] - 100s 621ms/step - loss: 0.7455 - accuracy: 0.7137
Epoch 7/10
161/161 [==============================] - 100s 620ms/step - loss: 0.7427 - accuracy: 0.7133
Epoch 8/10
161/161 [==============================] - 100s 619ms/step - loss: 0.7139 - accuracy: 0.7282
Epoch 9/10
161/161 [==============================] - 99s 615ms/step - loss: 0.6971 - accuracy: 0.7270
Epoch 10/10
161/161 [==============================] - 100s 623ms/ste

In [10]:

# Step 7: Predict Class for New Images
# Load your test images and preprocess them similarly to the training images.
# Then use model.predict() to get the predicted probabilities or classes.

# Step 8: Class Name Mapping
class_names = list(train_generator.class_indices.keys())

# Example usage of model.predict()
# test_image = load_and_preprocess_test_image('test_image.jpg')
# predictions = model.predict(test_image)
# predicted_class_index = tf.argmax(predictions, axis=1)
# predicted_class_name = class_names[predicted_class_index]


In [11]:
model.save("Railway-Track-Surface-Faults.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

def preprocess_image(image_path):
  img = load_img(image_path, target_size=(224, 224))
  img_array = img_to_array(img)
  img_array /= 255.0
  return np.expand_dims(img_array, axis=0)